# What generalizable patterns can we find about health care costs for smokers?

## Goals

This case will be focused on building the core mathematical foundations necessary for further study of data science. Up until this point, we have been focused on analyzing data from an *empirical* perspective - that is, working with tangible data points in Python. However, more advanced study of the data science arena (such as with hypothesis testing & inference, predictive modeling, and machine learning) often requires some *theoretical* background as well, where we work with abstractions of the truth that aren't completely bound by the data we have. This case will address both.

## Introduction

**Business Context.** You are an employee at a health insurance company. Your employer wants to better understand how their competition adjusts charges for smokers. You have been asked to investigate some historical data from this company to clarify the characteristics of smokers as part of this project.

**Business Problem.** Your manager has asked you to answer the following question: **"What is the payment, personal, and regional profile of a smoker?"**

**Analytical Context.** The relevant data is a public CSV file called `insurance.csv`, which contains information on various features which may affect health insurance charges.

In this case, you will use what you know about basic statistics, along with Python and relevant plotting libraries. You will focus on ensuring that you have chosen appropriate metrics and plots to convey your observations.

## Loading up the data

Let's load in the health insurance dataset in `data/insurance.csv`. The dataset consists of the following features:

* **age:** the person's age, in years
* **sex:** male or female
* **bmi:** the person's Body Mass Index (MBI)
* **children:** the number of children the person has
* **smoker:** whether the person is a smoker or not
* **region:** Northeast, Northwest, Southeast, Southwest
* **charges:** the total insurance charges for that person

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/insurance.csv")
df.head()

Let's use the ``info()`` method to identify the types of the variables in the dataset.

In [ ]:
df.info()

We can summarize the variables above as follows:

| **Type**       | **Columns**         |
|----------------|---------------------|
| **Continuous** | bmi, charges        |
| **Discrete**   | age, children       |
| **Nominal**    | sex, smoker, region |
| **Ordinal**    |                     |

## Summary statistics for each variable

Recall our original business problem - we want to ascertain the payment, personal, and regional profile of smokers in the US. As you learned in an earlier case, a good place to start is by looking at **summary statistics** of data on smokers - examples being the mean, median, and mode of a dataset. All three are central tendency measures, that is, they give us an idea of what the typical (most central) element looks like in a dataset for a given variable. Recall that:

* **Mean** is the sum of all observations for a particular variable over the number of observations for that variable
* **Median** is another measure of the data's "center" and gives the middlemost value when data is listed in ascending order (i.e. the 50th percentile)
* **Mode** gives us the most common value that appears in the data. It is most often used for categorical data

### Exercise 1:

Use the `mean()`, `mode()`, `median()` and `groupby()` methods to answer the following questions:

1. What is the mean charge of all cases?
2. Is the mean charge for people who smoke greater than that for non-smokers? What is the percentage difference?
3. Is smoking more prevalent in males or females within the dataset?
4. What is the median charge for male smokers as opposed to female smokers? What is the percentage difference?

**Answer.** One possible answer is shown below:

In [ ]:
df["charges"].mean()

In [ ]:
df[["smoker","charges"]].groupby("smoker").mean()

In [ ]:
smoker_means = df[["smoker","charges"]].groupby("smoker").mean()
smoker = (smoker_means.loc["yes"])[0]
non_smoker = (smoker_means.loc["no"])[0]
percent = (smoker-non_smoker)/non_smoker 
print("Smokers pay {:.2f}% more than non-smokers on average".format(percent*100))

In [ ]:
df[df["smoker"]=="yes"]["sex"].mode()

In [ ]:
df[df["smoker"]=="yes"][["charges","sex"]].groupby("sex").median()

In [ ]:
sex_charges = df[df["smoker"]=="yes"][["charges","sex"]].groupby("sex").mean()
female = (sex_charges.loc["female"])[0]
male = (sex_charges.loc["male"])[0]
percent = (male-female)/female 
print("Male smokers pay {:.2f}% more than female smokers on average".format(percent*100))

Within this dataset: (1) there is a mean charge of \\$13270.42 levied across all cases (both smokers and non-smokers); (2) the mean charge for smokers is \$32050.23, which is 280% higher than for non-smokers; (3) there are more male smokers than female smokers, as indicated by the mode; (4) the median charge for male smokers, \\$36085.22, is 7.70% higher than for female smokers, \\$28950.47.

Of course, as we already know from previous cases, these measures of central tendency only tell us part of the story. The actual individual data points are not all located at the mean, median, or mode, but *distributed around them*, that is, they show **variability**.

### Exercise 2:

Let's look at some aspects of the variability of insurance charges that smokers pay:

1. What is the difference between the largest and smallest charges that smokers pay?
2. What range of charges do the middle 50% of smokers pay? (that is, the charges between the 25th and 75th percentiles, also called the **interquartile range**)


**Hint:** You can filter the DataFrame to get only smokers and get the maximum value of the `charges` variable as follows:

```python
mn = df[df["smoker"] == "yes"]["charges"].min()
```

You can get the inter quartile range by using:

```python
iqr = charges.quantile(q=0.75) - charges.quantile(q=0.25)
```

**Answer.** One possible answer is shown below:

In [ ]:
# Range - difference of extreme values
mn = df[df["smoker"] == "yes"]["charges"].min()
mx = df[df["smoker"] == "yes"]["charges"].max()
rnge = mx - mn
print("Range: [${:.2f} , ${:.2f}]".format(mn, mx))

In [ ]:
# Interquartile range
charges = df[df["smoker"] == "yes"]["charges"]
iqr = charges.quantile(q=0.75) - charges.quantile(q=0.25)
print(
    "The charges for the middle 50% of smokers range from ${:.2f} to ${:.2f}.\n\nThis is an interquartile range of ${:.2f}.".format(
        charges.quantile(q=0.25), charges.quantile(q=0.75), iqr
    )
)

We see that the charges range from \\$12,829.46 - \\$63,770.43. The charges for the middle 50% of smokers range from \\$20,826.24 to \\$41,019.21. This is an interquartile range of \\$20,192.96.

There is a measure of variability that is very similar to the interquartile range, but possesses some additional important mathematical properties that make it useful for tasks beyond simple descriptive statistics - the **standard deviation**. The formula for the standard deviation is as follows:

$$
\sigma = \sqrt{\frac{\sum(x_i-\mu)^2}{N}}
$$

Let's break this down. First, the Greek letter $\sigma$ (lowercase *sigma*) is the conventional way to represent the standard deviation in mathematical formulas. It is equal to the square root of this term:

$$\frac{\sum(x_i-\mu)^2}{N}$$

What this term is doing might seem complicated, but it's not so bad. You first take each value in your dataset (that's $x_i$) and subtract the mean $\mu$ from it. For instance, if the mean is 5 and $x_i$ is 11, then you have to subtract 5 from 11. You then raise the resulting number to the second power and repeat the process for all the numbers in your dataset. Then, you sum up all these values (that's what the Greek letter $\Sigma$, uppercase *sigma*, means) and divide the sum by $N$, which is the number of elements in the dataset (we do this to make this quantity into an average).

The reason why $(x_i-\mu)$ is raised to the second power is because for values of $x_i$ that are less than the mean, this value is negative, so when you sum across all of these values, the negative values would cancel out the positive ones and give us zero! You can therefore think of the standard deviation as the average deviation from the mean to any point in the dataset, either to the left or to the right.

We take the square root of the whole thing to bring it back to the same units that our dataset uses. For instance, if you have a dataset of people's heights, it makes little sense to have an average deviation in square feet or square meters, so you need to take the square root to have it in feet or meters.

### Analysis of regional charges

Histograms can be used to compare different categorical variables against a common scale. This allows for a visual understanding of the spread of the data across these categories.

We can have a histogram for each region:

![Histograms region](data/images/histograms_region.png)

In [ ]:
regional_charges = df[["charges", "region"]].groupby("region")
regional_charges.describe()

The Southeast region appears to have the highest charges. It has the highest mean and its third quartile spreads to higher values than those for the other regions. 

### Exercise 3:

This is a table of the count of smokers per region as a proportion of the total number of smokers in the dataset. Why does the Southeast have higher charges than the other regions?

| Region 	| Percentage of smokers 	|
|-	|-	|
| Northwest 	| 21% 	|
| Northeast 	| 24% 	|
| Southeast 	| 33% 	|
| Southwest 	| 21% 	|



**Answer.** One possible answer is this:

From the table, we see that the Southeast has the highest smoker prevalence at 33%. This means that smokers are not evenly distributed across the regions. As this is also the region with the highest charges, and we know that smokers pay higher charges than non-smokers, it is likely that the higher prevalence of smokers in this region is the reason that the charges in this region are higher. Notice however that we can't directly assert that there is a *causal* relationship between being a smoker and higher charges; based on the data we have, we can only say that these variables are at most correlated (see [here](https://www.nature.com/articles/nmeth.3587) for an in-depth discussion of the difference between causation and correlation).

It is often necessary or useful to calculate the **probability** of some event $A$ occurring, which takes on a value from 0 to 1 and is denoted $P(A)$. A probability of 0 means that an event is certain to not occur, and a probability of 1 means that it is certain to occur. To calculate the probability of events occurring, we can use the following rules (here, $A$ and $B$ are two events): 

1. $\displaystyle P(A) = \frac {\textrm{# of outcomes when } A \textrm{ occurs}} {\textrm{# of total outcomes}}$
2. $P(A \textrm{ AND } B) = P(A) * P(B)$ (only applies to [independent events](https://www.statisticshowto.com/probability-and-statistics/dependent-events-independent/))
3. $P(A \textrm{ OR } B) = P(A) + P(B)$ (only applies to [mutually exclusive events](https://en.wikipedia.org/wiki/Mutual_exclusivity))

### Exercise 4:

The company is considering offering a lucky draw to give one client zero charges for the next month. They want to know how likely it is that a smoker is selected. Specifically:

1. If the company did one lucky draw for each region individually, what is the probability that the selected person in the region that smokes the most is a smoker?
2. If the company did one lucky draw for all regions together, what is the probability of a smoker from the region that smokes the most being selected?

**Answer.** One possible solution is given below:

In [ ]:
# Probability of a person in the Southeast being a smoker
se_smokers = df[df["region"] == "southeast"]["smoker"].value_counts()
p = se_smokers["yes"] / (se_smokers.sum())
print(p * 100)

In [ ]:
# Probability of a smoker from the Southeast being selected
p = se_smokers["yes"]/df.shape[0]
print("{:.2f}".format(p * 100))

The probability of a person in the Southeast being a smoker is the number of smokers divided by the total population in that region. Hence, a randomly selected person in that region would have a 25% chance of being a smoker.

To compute the probability of a smoker from the Southeast being selected, we need to divide the number of smokers in the Southeast by the total population across all regions. This yields that in this dataset there is a 6.8% chance that a randomly selected case would be a smoker from the Southeast.

## Relationships between variables

As we have seen in previous cases, properties of individual variables are important, but they are not the only important thing. Rather, data science often consists of the study of relationships between multiple variables. One way to give a quick numerical summary of a relationship between two variables is via the concept of **correlation**.

The **Pearson correlation coefficient** (also called Pearson's $R$) is commonly used for this purpose and has values in the range \[-1:1\]. A positive correlation ($R > 0$) means that as one variable increases in value, the common trend is that the other variable increases in value, while a negative correlation ($R < 0$) means that as one variable increases, the other decreases. If $R = 1$, then that means there is a perfect positive linear relationship between the two variables; if $R = -1$, there is a perfect negative linear relationship.

### Exercise 5:

Use `pandas` and the `.corr()` method to compute a correlation matrix to compare correlations between variables for smokers. Use this matrix to identify which variables are positively correlated, negatively correlated, and uncorrelated with the charges.

**Answer.** One possible solution is shown below:

In [ ]:
smokers = df[df["smoker"] == "yes"]
central = smokers[
    (smokers["charges"] >= charges.quantile(q=0.25))
    & (smokers["charges"] <= charges.quantile(q=0.75))
]
corr = central.corr()
corr.style.background_gradient(cmap="coolwarm").set_precision(2)

We can see that there is virtually no correlation between the number of children and charges, which indicates that there is no linear relationship between these two variables.

### Exercise 6:

These are scatterplots of the continuous variables present in the dataset. Explain what each of these means.

![Scatterplot of BMI and charges](data/images/scatter_bmi_charges.png)
![Scatterplot of age and charges](data/images/scatter_age_charges.png)


**Answer.** One possible answer is the following:

* The charges are fairly strongly positively correlated for BMI ($R=0.78$). This agrees with what we see on the scatter plot. We can also immediately see that there is some spread away from the central trend, which is why the $R$ value is not perfect.
* There is a small negative correlation between age and charges. This indicates that as age increases, charges decrease. The scatter plot clearly shows two different groups within the data. We see that after the age of 40, charges are generally lower. However, within each of these groups, `charge` seems to increase with `age`. This indicates that the correlation between these variables is not particularly indicative of a linear trend [if we don't account for the presence of these clusters](https://en.wikipedia.org/wiki/Simpson%27s_paradox).

## Probability theory and distributions

Thus far, we have been concerned with calculating statistical values which give insight into the data we already have; i.e. we have been taking a more *empirical* approach. However, we often have to be forward-looking and also predict the likelihood of future events, which is more *theoretical* in nature. We had a sampling of this in Exercise 4, but here we will talk more in detail about distributions and some of the basic theory around them.

Suppose we were to plot the distribution of BMIs for the smokers in our dataset — that would give us a histogram. But a histogram is entirely based on a limited sample of past data, and doesn't allow us to directly make predictions about the general smoker population, *independent of our specific dataset*. To do this, we need a **probability density function (PDF)**, which is a theoretical construct that helps us compute the probability of where any randomly chosen smoker's BMI would fall, and which is not strictly beholden to the limits of our dataset (caveat: In order to use a PDF as an aid for inference, ie., to make general claims about a population using data from a sample, we need to pair it with some other techniques that make sure that the sample is representative of the whole).

Notice that we didn't say that the PDF can directly tell us the probability that a randomly chosen smoker's BMI would be exactly some given value; say, 30. The reason for this is because BMI is a *continuous* value rather than a *discrete* value; it can take on any decimal value, so the probability that a person's BMI is *exactly* 30 is pretty much zero. It's virtually guaranteed that what we think is a 30 BMI is actually a 29.99 BMI, or a 30.01 BMI, or even a 30.001 BMI. Either way, finding a person with a BMI of exactly 30 is like trying to find a unicorn.

### Area under the curve

Because of this, it makes more sense to compute the probability that a smoker's BMI lies within some range; say, between 29 and 31. The PDF is perfectly made for this - specifically, if we plot the PDF on a standard 2-dimensional coordinate plane (with an $x$-axis and $y$-axis), this probability is the area of the region that lies under the PDF and above 0 (the $x$-axis), with $x$ values between 29 and 31. For those of you familiar with calculus, this is called the **area under the curve**.

One interesting consequence of this property of the PDF is that the total area under the PDF and above the $x$-axis, with no limits on what the $x$ values can be, has to be 1 (because the probability of an event has to lie between 0 and 1).

One of the most famous distributions is the **normal distribution**, for which the mean, median, and mode are all equal and which is symmetric around the median. This distribution has a high peak in the center and tapers off as we move towards the left and right ends.

### Exercise 7:

Here is a probability density function of the BMI of all smokers:

![PDF of BMI of smokers](data/images/PDF_bmi_smokers.png)

From this plot and the mean, median, and mode values, explain if this distribution is a good candidate for being perfectly normal.

**Answer.** One possible answer is shown below:

In [ ]:
mean = df[df["smoker"] == "yes"]["bmi"].mean()
median = df[df["smoker"] == "yes"]["bmi"].median()
mode = df[df["smoker"] == "yes"]["bmi"].mode()
mode = mode.values[0]

print(
    "The mean, median and mode are very similar: {:.2f}, {:.2f}, {}.".format(
        mean, median, mode
    )
)

The distribution is bell-shaped and is fairly symmetric around the mean, with a central peak and a "tail" on both sides. However, the right tail is longer than the left tail, and there is a slight dent to the right of the peak. The mean and median are very close, although the mode is slightly lower. Thus, this distribution is fairly close to normal, but not quite.

An ideal normal distribution looks like this:

!["A normal curve"](data/images/normal.png)

As you can see, there are some vertical lines overlaid on the curve. These represent rules of thumb which are *very* helpful to remember:

* Approximately 68% of samples in a normal distribution fall within one standard deviation of the mean
* Approximately 95% of samples fall within two standard deviations of the mean
* Approximately 99.7% of samples fall within three standard deviations of the mean

It is common practice to refer to a data point's number of standard deviations from the mean as its **$z$-score**.

Relating this back to our discussion about area under the curve, the above rules of thumb can be interpreted as:

* The area under the normal distribution PDF and above the $x$-axis, with a $z$-score between -1 and 1 is approximately 0.68.
* The area under the normal distribution PDF and above the $x$-axis, with a $z$-score between -2 and 2 is approximately 0.95.
* The area under the normal distribution PDF and above the $x$-axis, with a $z$-score between -3 and 3 is approximately 0.997.

### Exercise 8:

Assume for now that the distribution of BMIs is perfectly normal. Compute the following:

1. What is the probability that a smoker has a BMI of more than 30 (the threshold for obesity)?
2. What is the probability that a smoker has a BMI of more than 37?
3. What range of values have a 5% or lower probability of being observed if you randomly sample one person from this distribution?

**Answer.** One possible answer is shown below:

In [ ]:
# mean and std
smoker_bmi = df[df["smoker"] == "yes"]["bmi"]
mean_bmi = smoker_bmi.mean()
std_bmi = smoker_bmi.std()

print("Mean: {:.2f}\nStd: {:.2f}\n".format(mean_bmi, std_bmi))

In [ ]:
print(
    "50 % of observations lie to either side of the mean, thus 50 % probability that BMI > 30"
)

In [ ]:
# Probability of BMI more than 37 - properties of a normal dist
print("One std above the mean: {:.1f}".format(mean_bmi + std_bmi))

P = 0.68 / 2 + 0.5
print("{:.2f}".format(P))
print("P > 37 = {:.2f} ".format((1 - P) * 100))

In [ ]:
# 2 std of mean - 95% of observations in 2 std of mean.
lower_lim = mean_bmi - 2 * std_bmi
upper_lim = mean_bmi + 2 * std_bmi

print("BMI < {:.2f} and BMI > {:.2f}".format(lower_lim, upper_lim))

The mean is 30.71, which is fairly close to 30. We are looking for the probability of the BMI being above 30. 50% of the samples fall to either side of the mean in a normal distribution, which means that the probability of sampling a BMI greater than 30 is roughly 50%.

One standard deviation from the mean is 30.71 + 6.32 = 37.03, which is very close to 37; thus, we are looking for the probability of sampling a BMI greater than 1 standard deviation above the mean. We know that 68% of observations fall within 1 standard deviation. Because the distribution is symmetric, we know that half of these are less than the mean and half are more than the mean. Thus, the probability is (1 - 0.68)/2, which is 16%.

95% of observations fall within two standard deviations from the mean. This means that the remaining 5% of samples fall beyond 2 standard deviations, which gives the ranges of BMI < 18.07 ~ 18 and BMI > 43.35. 

## Conclusions

From our analysis, we have seen that smokers pay substantially higher fees than non-smokers. Fees for smokers are also strongly correlated with BMI, particularly above the threshold for obesity. It seems that smoking is more common in males than females and that it is particularly prevalent in the Southeast region, which could account for the higher median charge in that region.

## Takeaways

In this case, you saw that we can analyze historical data to understand the trends within it, and use those trends and some theoretical understanding of probability and statistics to predict the likelihood of future events. Key points include:

1. Summary statistics can be used to quickly indicate where the "center" of a dataset is, or how spread out the dataset is.
3. We can investigate correlations between variables. This is especially useful during exploratory data analysis, and when (as you will see in future cases) building models that predict the value of one variable of interest based on several others.
3. The probability density function is what allows us to move from the fully empirical world of histograms, which are an imperfect and limited representation of the entire population, to modeling and inferring characteristics of that population. The area under the PDF and above 0 between two specific $x$-values $a$ and $b$ gives the probability that a randomly chosen sample from that population will have an $x$-value for that variable between $a$ and $b$.
4. The normal distribution is a particularly special PDF which is perfectly symmetric and with its mean, median, and mode all equal, and is a pretty good fit to many natural phenomena.

## Attribution

"Standard deviation diagram", M.W. Toews, 7 April 2007, Creative Commons Attribution 2.5 Generic license, https://commons.wikimedia.org/wiki/File:Standard_deviation_diagram.svg